<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_HR_gr2/blob/Kulinichenko-Alexander-folder/Kulinichenko%20Alexander%5C%D0%A1%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B0_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_2_%D0%B3%D1%80%D1%83%D0%BF%D0%BF%D0%B0_01_04_2023_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B1%D0%B0%D0%B7%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Работа с массивами данных
import numpy as np 

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

#Изучение датасета

In [2]:

# Загрузка файла из облака в colab
pact_file = "https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_HR/datasets/%D0%94%D0%B0%D1%82%D0%B0%D0%A1%D0%B5%D1%82%20%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20581%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5.xlsx"
gdown.download(pact_file, None, quiet=True)
data_xsl = pd.read_excel(pact_file, sheet_name='Лист1')
data_xsl.head()

,Unnamed: 0,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
0,0,2023-02-23 00:00:00,Bessonova Mariya,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да
1,1,2023-01-11 00:00:00,Ефремова Екатерина Вячеславовна,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да
2,2,2023-02-27 00:00:00,Фролов Артур Михайлович,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да
3,3,2023-01-11 00:00:00,Хозаев Расул Шагаманович,"Мужчина, 32 года, родился 7 декабря 1990",не указана,Growth Product Manager,"Москва, м. Молодежная, Хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",Опыт работы —8 лет 10 месяцев,WeInvest,Product Manager (team leader),"Магистр, 2014, Московский государственный техн...",24 января 2023 в 12:07,да
4,4,2023-01-11 00:00:00,Шадрин Олег Михайлович,"Мужчина, 26 лет, родился 21 февраля 1996",150 000 руб.,Growth Product Manager,"Пермь, Хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —5 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product/Project manager,"Высшее, 2021, Российская академия народного хо...",6 февраля 2023 в 11:18,да


In [3]:
#количество строк и столбцов
data_xsl.shape

(579, 15)

In [ ]:
data_xsl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Unnamed: 0                       579 non-null    int64 
 1   Дата отклика                     579 non-null    object
 2   ФИО                              579 non-null    object
 3   Пол, возраст                     579 non-null    object
 4   ЗП                               579 non-null    object
 5   Ищет работу на должность         579 non-null    object
 6   Город                            579 non-null    object
 7   Занятость                        579 non-null    object
 8   График                           579 non-null    object
 9   Опыт ()                          579 non-null    object
 10  Последнее/нынешное место работы  579 non-null    object
 11  Последняя/нынешняя должность     579 non-null    object
 12  Образование и ВУЗ                579

##Предварительные выводы:
0. Номер по порядку
1. дата отклика
2. ФИО
3. Пол, возраст.  ЗАМЕЧАНИЕ: Не у всех заполнен возраст и дата рождения
4. ЗП. ЗАМЕЧАНИЕ: К некоторых не указана
5. Ищет работу на должность. ЗАМЕЧАНИЕ: Всего 8 позиций
6. Город. ЗАМЕЧАНИЕ: У некоторых всесто города указана страна. так же в этом поле инвормация готов, неготов к переезду, командировкам
7. Занятость. ЗАМЕЧАНИЕ: Всего 20 разновидностей заполнения (из 5 возможных)типа: полная занятость, частичная занятость, проектная работа, стажировка, волонтерство
8. График.  ЗАМЕЧАНИЕ: Всего 30 вариантов заполнения ( из 5 возможных) типа вахтовый метод, удаленная работа (удаленная), гибкий график, полный день, сменный график. Также ошибки типа: "сменный графи", "удаленная работа\n\n". Из за чего увеличиваются варианты заполнения.
9. Опыт (). ЗАМЕЧАНИЕ: указывается количество лет и месяцев
10. Последнее/нынешное место работы. 
11. Последняя/нынешняя должность.
12. Образование и ВУЗ. 
13. Обновление резюме.
14. Подходит - да/нет.  ЗАМЕЧАНИЕ: Да - 11, НЕТ - 489, Не запполнено - 79. По данному столбцу база разбалансировна.

В техническом задании есть ссылки на блоки “Ключевые навыки”, “Опыт работы”, “Обо мне”, которые отсутствуют в исходной базе данных. 

In [5]:
data_xsl_ = data_xsl.groupby(['Ищет работу на должность']).count()
#data_xsl_ = data_xsl.groupby(['Занятость']).count()
#data_xsl_ = data_xsl.groupby(['График']).count()
#data_xsl_ = data_xsl.groupby(['Опыт ()']).count()
#data_xsl_ = data_xsl.groupby(['Последняя/нынешняя должность']).count()
#data_xsl_ = data_xsl.groupby(['Подходит - да/нет']).count()
data_xsl_

,Unnamed: 0,Дата отклика,ФИО,"Пол, возраст",ЗП,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
Ищет работу на должность,,,,,,,,,,,,,,
Growth Product Manager,198,198,198,198,198,198,198,198,198,198,198,198,198,184
HR-специалист,115,115,115,115,115,115,115,115,115,115,115,115,115,115
Product Manager в команду HR,25,25,25,25,25,25,25,25,25,25,25,25,25,13
Product Manager в команду Финансового департамента,36,36,36,36,36,36,36,36,36,36,36,36,36,25
Product Marketing Manager в Growth Hacking команду,14,14,14,14,14,14,14,14,14,14,14,14,14,6
Product manager в Growth Hacking команду,77,77,77,77,77,77,77,77,77,77,77,77,77,56
Project manager,21,21,21,21,21,21,21,21,21,21,21,21,21,8
Директор медицинской клиники,93,93,93,93,93,93,93,93,93,93,93,93,93,93


In [ ]:
data_xsl.describe(include=['O'])

,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
count,579,579,579,579,579,579,579,579,579,579,579,579,579,500
unique,89,507,443,44,8,209,20,30,252,486,358,488,547,2
top,2023-02-22 00:00:00,ФИО скрыто соискателем,Женщина,не указана,Growth Product Manager,"Уфа, Не готова к переезду, не готова к команди...",полная занятость,полный день,Опыт работы —6 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product manager,Среднее образование,31 января 2023 в 09:52,нет
freq,48,7,41,288,198,69,342,202,10,12,42,15,3,489


# Парсим базу

In [10]:
data_xsl = data_xsl.drop(data_xsl.columns[0],axis = 1) # Убираем первый столбец с продублированными индексами. Он нам не нужен

In [11]:

# Замена концов строк на пробелы, удаление символа с кодом 0xA0
# обрезка краевых пробелов, приведение к нижнему регистру
def purify(x):
    if isinstance(x, str):                # Если значение - строка:
 #       x = x.replace('\n', ' ').replace('\xa0', '').strip().lower()
        x = x.lower()
    return x

data_xsl = data_xsl.applymap(purify)
data_xsl.head()              

,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
0,2023-02-23 00:00:00,bessonova mariya,женщина,90 000 руб.,growth product manager,"индонезия, не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,опыт работы —5 лет 10 месяцев,pt mybali app group,менеджер по продукту,"высшее, 2023, нетоло́гия, 2014, университет ро...",23 февраля 2023 в 16:30,да
1,2023-01-11 00:00:00,ефремова екатерина вячеславовна,женщина,не указана,growth product manager,"москва, готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",опыт работы —11 лет 11 месяцев,банк открытие,product owner трайб сбережения и инвестиции,"высшее, 2022, geekbrains, 2010, московский худ...",8 февраля 2023 в 09:15,да
2,2023-02-27 00:00:00,фролов артур михайлович,"мужчина, 33 года, родился 23 августа 1989",150 000 руб.,growth product manager,"санкт-петербург, готов к переезду: москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",опыт работы —7 лет 2 месяца,dikiy llc,директор по маркетингу и бренд-менеджер,"высшее, 2013, санкт-петербургский государствен...",28 февраля 2023 в 09:41,да
3,2023-01-11 00:00:00,хозаев расул шагаманович,"мужчина, 32 года, родился 7 декабря 1990",не указана,growth product manager,"москва, м. молодежная, хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",опыт работы —8 лет 10 месяцев,weinvest,product manager (team leader),"магистр, 2014, московский государственный техн...",24 января 2023 в 12:07,да
4,2023-01-11 00:00:00,шадрин олег михайлович,"мужчина, 26 лет, родился 21 февраля 1996",150 000 руб.,growth product manager,"пермь, хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",опыт работы —5 лет 9 месяцев,индивидуальное предпринимательство / частная п...,product/project manager,"высшее, 2021, российская академия народного хо...",6 февраля 2023 в 11:18,да


In [14]:
# Разбор значений пола (вариант от елены соколовой измененный для проверки если нет указания на мужчина это или женщина)
def extract_gender(arg):
    if 'женщина' in arg.lower():
        return 0
    elif 'мужчина' in arg.lower():
        return 1
    else:
        return "2"    # указатель что не указано мужчина или женщина

columns =['Пол', 'возраст']
data_clean = pd.DataFrame(data=None, index=None, columns=columns, dtype=None)
data_clean['Пол'] = data_xsl['Пол, возраст'].apply(extract_gender)

#data_clean
data_clean['Пол'].value_counts()  
#Если нет двоек то все хорошо


1    302
0    277
Name: Пол, dtype: int64

In [13]:
# Разбор значений пола, возраста

base_update_year = 2023

def extract_sex_age_years(arg):
    # Ожидается, что значение содержит "мужчина" или "женщина"
    # Если "мужчина" - результат 1., иначе 0.
    sex = 1. if 'муж' in purify(arg) else 0.

    
    return sex

columns =['Пол', 'возраст']
data_clean = pd.DataFrame(data=None, index=None, columns=columns, dtype=None)
data_clean['Пол'] = data_xsl['Пол, возраст'].apply(extract_sex_age_years)

data_clean
data_clean['Пол'].value_counts()

1.0    302
2.0    277
Name: Пол, dtype: int64

In [8]:
# Разбор значений пола, возраста

base_update_year = 2023

def extract_sex_age_years(arg):
    # Ожидается, что значение содержит "мужчина" или "женщина"
    # Если "мужчина" - результат 1., иначе 0.
    sex = 1. if 'муж' in arg else 0.

    try:
        # Выделение года и вычисление возраста
        years = base_update_year - int(re.search(r'\d{4}', arg)[0])

    except (IndexError, TypeError, ValueError):
        # В случае ошибки год равен 0
        years = 0

    return sex, years

columns =['Пол', 'возраст']
data_clean = pd.DataFrame(data=None, index=None, columns=columns, dtype=None)
data_clean['Пол', 'возраст'] = data_xsl['Пол, возраст'].apply(extract_sex_age_years)

#data_clean['Пол'].value_counts()
data_clean

,Пол,возраст,"(Пол, возраст)"
0,NaN,NaN,"(0.0, 0)"
1,NaN,NaN,"(0.0, 0)"
2,NaN,NaN,"(0.0, 34)"
3,NaN,NaN,"(0.0, 33)"
4,NaN,NaN,"(0.0, 27)"
...,...,...,...
574,NaN,NaN,"(0.0, 25)"
575,NaN,NaN,"(0.0, 43)"
576,NaN,NaN,"(0.0, 40)"
577,NaN,NaN,"(0.0, 39)"


In [ ]:
# Выделение подстроки вида ДД.ММ.ГГГГ и возвращение значения года

def extract_year(x):
    try: 
        return int(re.search(r'\d\d.\d\d.(\d{4})', x)[1])   # Ожидается строка вида 'dd.mm.yyyy ...'
  
    except (IndexError, TypeError, ValueError):
        return 0     